In [1]:
import pandas as pd
import pyodbc
username = 'vigrose'
password_aims = 'DQA8K@T7pXEqEt'
password_edw = 'Ravenclaw~10946'

In [2]:
s = "DSN=aims_prod; UID={}; PWD={}".format(username, password_aims)
informix = pyodbc.connect(s)

In [3]:
w = "DSN=PRDDW; UID={}; PWD={}".format(username, password_edw)
AMAEDW = pyodbc.connect(w)

In [4]:
entity_query = \
    '''
    SELECT DISTINCT 
    M.ME,
    E.ENTITY_ID
    FROM 
    AMAEDW.PARTY_ID_TO_ME_VW M, AMAEDW.PARTY_ENTITY_VW E
    WHERE 
    M.PARTY_ID=E.PARTY_ID
    '''
ethnic_query = \
    '''
    SELECT 
    P.ETHNICITY,
    P.ENTITY_ID
    FROM informix.PERSON_ET P
    WHERE
    P.ACTIVE_IND = 'Y'
    ;
    '''
code_query = \
    '''
    SELECT 
    P.ETHNICITY,
    P.DESCRIPTION
    FROM informix.ETHNICITY_ER P
    WHERE
    P.ACTIVE_IND = 'Y'
    ;
    '''

ETHNICITY_OLD = pd.read_sql(con=informix, sql=ethnic_query)
me_entity = pd.read_sql(con=AMAEDW, sql=entity_query)
OLD_CODES = pd.read_sql(con=informix, sql=code_query)
ETHNICITY_OLD['ENTITY_ID']=ETHNICITY_OLD.entity_id.astype(str)
ethnic_old = pd.merge(ETHNICITY_OLD, me_entity, on='ENTITY_ID', how='left')

In [5]:
entity_query = \
    '''
    SELECT DISTINCT 
    M.ME,
    E.ENTITY_ID
    FROM 
    AMAEDW.PARTY_ID_TO_ME_VW M, AMAEDW.PARTY_ENTITY_VW E
    WHERE 
    M.PARTY_ID=E.PARTY_ID
    '''
ethnic_query = \
    '''
    SELECT 
    P.ETHNICITY,
    P.ENTITY_ID
    FROM informix.PERSON_ET P
    WHERE
    P.ACTIVE_IND = 'Y'
    ;
    '''
ETHNICITY_OLD = pd.read_sql(con=informix, sql=ethnic_query)
me_entity = pd.read_sql(con=AMAEDW, sql=entity_query)

ETHNICITY_OLD['ENTITY_ID']=ETHNICITY_OLD.entity_id.astype(str)
ethnic_old = pd.merge(ETHNICITY_OLD, me_entity, on='ENTITY_ID', how='left')

In [6]:
ETHNICITY_OLD = pd.read_sql(con=informix, sql=sql_query)
ETHNICITY_NEW = pd.read_csv('../../Data/MasterfileCore/AAMC_ETHNICITY_110520.csv')
me_entity = pd.read_sql(con=AMAEDW, sql=entity_query)
codes = pd.read_excel('../../Data/MasterfileCore/Race Ethnicity AAMC Code Values.xlsx')

NameError: name 'sql_query' is not defined

In [ ]:
OLD_CODES = pd.read_sql(con=informix, sql=code_query)

In [7]:
informix.close()

In [ ]:
ETHNICITY_OLD['ENTITY_ID']=ETHNICITY_OLD.entity_id.astype(str)
ethnic_old = pd.merge(ETHNICITY_OLD, me_entity, on='ENTITY_ID', how='left')

In [ ]:
def fix_me(me_list):
    nums = []
    for num in me_list:
        num = str(num)
        num = num.replace('.0', '')
        if len(num) == 10:
            num = '0' + num
        elif len(num) == 9:
            num = '00' + num
        elif len(num) == 8:
            num = '000' + num
        nums.append(num)
    return nums

In [ ]:
ETHNICITY_NEW['ME']=fix_me(ETHNICITY_NEW.me)

In [ ]:
ethnic_new = pd.merge(ETHNICITY_NEW, me_entity, on='ME', how='left')

In [ ]:
ethnic_new['IDENTITY_CAT_CD']=[x[0] for x in ethnic_new.identity_detail_cd]

In [48]:
ethnic_new_2 = ethnic_new.drop_duplicates(['me','IDENTITY_CAT_CD'])

In [50]:
ethnic_new_race = ethnic_new_2.drop_duplicates('me', keep=False)

In [53]:
mixed = ethnic_new_2[ethnic_new_2.me.isin(ethnic_new_race.me)==False].drop_duplicates('me')

In [56]:
mixed['IDENTITY_CAT_CD']='Mixed Race/Ethnicity'

In [58]:
ethnic_new_3 = pd.concat([ethnic_new_race, mixed])

In [59]:
all_that = pd.merge(ethnic_new_3, ethnic_old, on='ENTITY_ID', how='outer')

In [64]:
all_that = all_that.fillna('None')

In [67]:
CODE_LIST = []
for row in all_that.itertuples():
    CODE = 'None'
    if row.IDENTITY_CAT_CD != 'None':
        CODE = row.IDENTITY_CAT_CD
    elif row.ethnicity != 'None':
        CODE = row.ethnicity
    CODE_LIST.append(CODE)

In [69]:
all_that['CODE'] = CODE_LIST

In [97]:
all_that[(all_that.ME!='None')][['ME','CODE']].to_csv('../../Data/MasterfileCore/PhysicianEthnicity.csv', index=False)

In [85]:
mes =[]
for row in all_that.itertuples():
    if row.ME_x != 'None':
        ME = row.ME_x
        if row.ME_y != 'None' and row.ME_y != row.ME_x:
            print(row.ME_y)
            print(row.ME_x)
            print('')
    elif row.ME_y != 'None':
        ME = row.ME_y
    else:
        ME = 'None'
    mes.append(ME)

In [87]:
all_that['ME']=mes

In [99]:
codes

,IDENTITY_DETAIL_CD,IDENTITY_CAT_CD,IDENTITY_DESC,REC_STATUS,UPD_DT,ROW_COMMENTS
0,A00,A,Asian (Category),A,2013-02-21 00:00:00,NaN
1,A01,A,Other Asian,A,2013-02-21 00:00:00,NaN
2,A02,A,Bangladeshi,A,2013-02-21 00:00:00,NaN
3,A03,A,Cambodian,A,2013-02-21 00:00:00,NaN
4,A04,A,Chinese,A,2013-02-21 00:00:00,NaN
5,A05,A,Filipino,A,2013-02-21 00:00:00,NaN
6,A06,A,Indian,A,2013-02-21 00:00:00,NaN
7,A07,A,Indian/Pakistani,I,2013-02-21 00:00:00,Deprecated
8,A08,A,Indonesian,A,2013-02-21 00:00:00,NaN
9,A09,A,Japanese,A,2013-02-21 00:00:00,NaN


In [106]:
OLD_CODES

,ethnicity,description
0,1,White
1,2,Black or African American
2,3,"Hispanic, Latino, or Spanish Origin"
3,4,Asian
4,5,Other
5,6,American Indian
6,7,American Indian or Alaskan Native
7,9,Unknown


In [107]:
all_that

,me,identity_detail_cd,beginning_date,reported_date,end_date,preffered_status,effective_status,insert_user_id,insert_dtm,update_user_id,update_dtm,opt_lock,ME_x,ENTITY_ID,IDENTITY_CAT_CD,ethnicity,entity_id,ME_y,CODE,ME
0,1.0201e+08,W00,08/27/2014,08/27/2014,None,Y,A,etlaims,04/16/2020 05:02:49 PM,etlaims,04/16/2020 05:02:49 PM,0,00102010159,1236907,W,1,1.23691e+06,00102010159,W,00102010159
1,1.0201e+08,A00,08/18/2015,08/17/2016,None,Y,A,etlaims,04/16/2020 05:02:49 PM,etlaims,04/16/2020 05:02:49 PM,0,00102010396,1237219,A,3,1.23722e+06,00102010396,A,00102010396
2,1.02013e+08,B00,03/20/2015,03/01/2016,None,Y,A,etlaims,04/16/2020 05:02:49 PM,etlaims,04/16/2020 05:02:49 PM,0,00102012771,1238857,B,2,1.23886e+06,00102012771,B,00102012771
3,1.0202e+08,W00,08/07/2014,08/17/2016,None,Y,A,etlaims,04/16/2020 05:02:49 PM,etlaims,04/16/2020 05:02:49 PM,0,00102020146,14850143,W,1,1.48501e+07,00102020146,W,00102020146
4,1.02022e+08,B00,04/25/2016,08/17/2017,None,Y,A,etlaims,04/16/2020 05:02:49 PM,etlaims,04/16/2020 05:02:49 PM,0,00102021797,15050161,B,2,1.50502e+07,00102021797,B,00102021797
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3437587,None,None,None,None,None,None,None,None,None,None,None,None,None,63398855,None,None,6.33989e+07,None,None,None
3437588,None,None,None,None,None,None,None,None,None,None,None,None,None,63398868,None,None,6.33989e+07,None,None,None
3437589,None,None,None,None,None,None,None,None,None,None,None,None,None,63398881,None,None,6.33989e+07,None,None,None
3437590,None,None,None,None,None,None,None,None,None,None,None,None,None,63398894,None,None,6.33989e+07,None,None,None


In [108]:
phys_eth = all_that[(all_that.ME!='None')][['ME','CODE']]

In [111]:
OLD_CODES

,ethnicity,description
0,1,White
1,2,Black or African American
2,3,"Hispanic, Latino, or Spanish Origin"
3,4,Asian
4,5,Other
5,6,American Indian
6,7,American Indian or Alaskan Native
7,9,Unknown


In [110]:
codes

,IDENTITY_DETAIL_CD,IDENTITY_CAT_CD,IDENTITY_DESC,REC_STATUS,UPD_DT,ROW_COMMENTS
0,A00,A,Asian (Category),A,2013-02-21 00:00:00,NaN
1,A01,A,Other Asian,A,2013-02-21 00:00:00,NaN
2,A02,A,Bangladeshi,A,2013-02-21 00:00:00,NaN
3,A03,A,Cambodian,A,2013-02-21 00:00:00,NaN
4,A04,A,Chinese,A,2013-02-21 00:00:00,NaN
5,A05,A,Filipino,A,2013-02-21 00:00:00,NaN
6,A06,A,Indian,A,2013-02-21 00:00:00,NaN
7,A07,A,Indian/Pakistani,I,2013-02-21 00:00:00,Deprecated
8,A08,A,Indonesian,A,2013-02-21 00:00:00,NaN
9,A09,A,Japanese,A,2013-02-21 00:00:00,NaN


In [123]:
ids= []
for row in phys_eth.itertuples():
    if row.CODE == 'W':
        id_code = '1'
    elif row.CODE == 'B':
        id_code = '2'
    elif row.CODE == 'H':
        id_code = '3'
    elif row.CODE == 'A':
        id_code = '4'
    elif row.CODE == 'X':
        id_code = '5'
    elif row.CODE == 'I':
        id_code = '7'
    elif row.CODE == 'P':
        id_code = '8'
    elif row.CODE == 'U' or row.CODE == 'Z' or row.CODE == 'None':
        id_code = '9'
    else:
        id_code = row.CODE
    ids.append(id_code)

In [124]:
phys_eth['ethnicity']=ids

In [117]:
phys_eth

,ME,CODE,IDENTITY_DETAIL_CODE,ethnicity
0,00102010159,W,W,1
1,00102010396,A,A,4
2,00102012771,B,B,2
3,00102020146,W,W,1
4,00102021797,B,B,2
...,...,...,...,...
3436415,00175210407,None,None,None
3436576,03575240291,None,None,None
3436653,05175220188,None,None,None
3436690,03575240304,None,None,None


In [127]:
new_df = pd.DataFrame([[8, 'Native Hawaiian or Other Pacific Islander'], ['Mixed Race/Ethnicity', 'Mixed Race/Ethnicity']], columns = ['ethnicity','description'])
codess = OLD_CODES.append(new_df)

In [125]:
phys_eth.groupby('ethnicity').count()

,ME,CODE,IDENTITY_DETAIL_CODE
ethnicity,,,
,108556,108556,108556
0,253797,253797,253797
1,832538,832538,832538
2,64620,64620,64620
3,70363,70363,70363
4,230978,230978,230978
5,21485,21485,21485
6,645,645,645
7,1900,1900,1900


In [132]:
pd.merge(phys_eth, codess, on='ethnicity')[['ME', 'description']].rename(columns = {'ME':'medical_education_number','description':'race_ethnicity'}).to_csv("../../Data/MasterfileCore/PhysicianRaceEthnicity.csv", index=False)